This iPyNotebook will run the dilutions and the IPTG induction of cells for the tRNA depletion experiment.

In [21]:
import opentrons.simulate
import os
from rich import print
import random

protocol_file = 'complete_protocol.py'
pre_data = 'day_2_pre.csv'
iptg_volume = 6 # In microliters. 6ul of 0.00625M IPTG is 0.01875mM IPTG in 200ul total

First, we'll define the locations of each cell on the plate. 

In [22]:
cell_locations = {
    'B2': ['GFP10', 'RBS0.125'],
    'C2': ['GFP25', 'RBS0.125'],
    'D2': ['GFP50', 'RBS0.125'],
    'E2': ['GFP75', 'RBS0.125'],

    'B3': ['GFP10', 'RBS0.25'],
    'C3': ['GFP25', 'RBS0.25'],
    'D3': ['GFP50', 'RBS0.25'],
    'E3': ['GFP75', 'RBS0.25'],

    'B4': ['GFP10', 'RBS0.5'],
    'C4': ['GFP25', 'RBS0.5'],
    'D4': ['GFP50', 'RBS0.5'],
    'E4': ['GFP75', 'RBS0.5'],

    'B5': ['GFP10', 'RBS1'],
    'C5': ['GFP25', 'RBS1'],
    'D5': ['GFP50', 'RBS1'],
    'E5': ['GFP75', 'RBS1'],

    'B6': ['GFP10', 'RBS2'],
    'C6': ['GFP25', 'RBS2'],
    'D6': ['GFP50', 'RBS2'],
    'E6': ['GFP75', 'RBS2'],

    'B7': ['GFP10', 'RBS4'],
    'C7': ['GFP25', 'RBS4'],
    'D7': ['GFP50', 'RBS4'],
    'E7': ['GFP75', 'RBS4'],

    'B8': ['blank', 'blank'],
    'C8': ['blank', 'blank'],
    'D8': ['blank', 'blank'],
    'E8': ['blank', 'blank'],
}

Now, we'll assign each preconditioning well to two experimental wells.

In [23]:
import random
random.seed('day 2')

valid_rows = list("BCDEFG")
valid_columns = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11']

valid_wells = [f"{row}{column}" for row in valid_rows for column in valid_columns]

wells_with_cells = [well for well in cell_locations if cell_locations[well] != ['blank', 'blank']]

final_positions = {}

for well in wells_with_cells:
    position_1 = random.choice(valid_wells)
    position_1_index = valid_wells.index(position_1)
    del(valid_wells[position_1_index])
    position_2 = random.choice(valid_wells)
    position_2_index = valid_wells.index(position_2)
    del(valid_wells[position_2_index])

    final_positions[position_1] = [well, cell_locations[well][0], cell_locations[well][1]]
    final_positions[position_2] = [well, cell_locations[well][0], cell_locations[well][1]]

# Fill the rest of the valid positions with blanks

for well in valid_wells:
    final_positions[well] = final_positions.get(well, ['blank', 'blank', 'blank'])


Next, we'll calculate the appropriate well dilutions for the experiment using the data from the plate reader.

In [24]:
import pandas as pd

def parse_platereader(filename):
    """Parse the output of a Teccan plate reader and make the data tidy"""
    with open(filename, 'r') as file:
        lines = file.readlines()
    lines = [line.strip() for line in lines]
    dataframe = pd.DataFrame(columns=['time', 'well', 'label', 'value', 'temperature'])
    datasets = []
    reading_dataset = [None, None]
    start_time = None
    for i, line in enumerate(lines):
        line = line.split(',')
        if line[0].startswith('End Time:'):
            continue
        elif line[0].startswith('Start Time:'):
            start_time = i
        elif start_time and any(line) > 0 and not any(reading_dataset):
            reading_dataset[0] = i
        elif start_time and any(line) == 0 and any(reading_dataset):
            reading_dataset[1] = i # Exclusive
            datasets.append(reading_dataset)
            reading_dataset = [None, None]

    def filter_function(test_value):
        if test_value == "":
            return False
        else: 
            return True

    for dataset in datasets:
        data = lines[dataset[0]:dataset[1]]
        label = data.pop(0).split(',')[0]

        if data[0].startswith('Cycle'):
            cycle = data.pop(0).split(',')[1:]

        times = list(filter(filter_function, data.pop(0).split(',')[1:]))
        temperature = list(filter(filter_function, data.pop(0).split(',')[1:]))
        data = list(filter(filter_function, [line.split(',') for line in data]))

        for line in data:
            well = str(line.pop(0))
            line = list(filter(filter_function, line))
            for i, value in enumerate(line):
                adding_frame = pd.DataFrame({
                    'time': float(times[i]),
                    'well': well,
                    'label': label,
                    'value': float(value),
                    'temperature': float(temperature[i]),
                    }, index=[0])


                dataframe = pd.concat([dataframe, adding_frame])

    return dataframe

platereader_file = pre_data
platereader_data = parse_platereader(platereader_file)


target_od600 = 0.0234506  # This is equivilant to 0.2 OD600 on our 1cm spectrophotometer

blank_wells = [x for x in cell_locations.keys() if 'blank' in cell_locations[x]]

cell_volumes = {}
lb_volumes = {}


# Get the latest timepoint from the platereader data
latest_timepoint = platereader_data['time'].max()

# Set the OD600 column to be a float


# Get the latest OD600 values for the blank wells
blank_od600 = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['well'].isin(blank_wells)) & (platereader_data['label'].isin(['OD600'])), 'value'].mean()

# Get all the wells that are used in the protocol
occupied_wells = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['label'].isin(['OD600'])), 'well']
# Remove the blank wells
occupied_wells = [well for well in occupied_wells if well not in blank_wells]

for start_well in occupied_wells:
    well_od600 = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['well'] == start_well) & (platereader_data['label'].isin(['OD600'])), 'value'].mean()
    well_od600 = well_od600 - blank_od600

    cell_volume = (target_od600 / well_od600) * 200
    if cell_volume > (200-iptg_volume):
        cell_volume = (200-iptg_volume)
    if cell_volume < 0:
        cell_volume = 0
    lb_volume = (200-iptg_volume) - cell_volume

    cell_volumes[start_well] = [start_well, cell_volume]
    lb_volumes[start_well] = lb_volume


/tmp/ipykernel_21269/214353169.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, adding_frame])


Next, we associate the calculated cell volumes and LB volumes with the final well positions

In [25]:
for well in final_positions:
    source_well = final_positions[well][0]

    if source_well == 'blank':
        final_positions[well].append(0)
        final_positions[well].append((200-iptg_volume))
    else:
        final_positions[well].append(cell_volumes[source_well][1])
        final_positions[well].append(lb_volumes[source_well])

print(final_positions)

{
    'D7': ['B2', 'GFP10', 'RBS0.125', 16.484034820387237, 177.51596517961275],
    'E9': ['B2', 'GFP10', 'RBS0.125', 16.484034820387237, 177.51596517961275],
    'D6': ['C2', 'GFP25', 'RBS0.125', 15.157534678396281, 178.84246532160373],
    'B6': ['C2', 'GFP25', 'RBS0.125', 15.157534678396281, 178.84246532160373],
    'E5': ['D2', 'GFP50', 'RBS0.125', 17.42729169118082, 176.5727083088192],
    'B10': ['D2', 'GFP50', 'RBS0.125', 17.42729169118082, 176.5727083088192],
    'D11': ['E2', 'GFP75', 'RBS0.125', 16.647865131322867, 177.35213486867713],
    'B5': ['E2', 'GFP75', 'RBS0.125', 16.647865131322867, 177.35213486867713],
    'B11': ['B3', 'GFP10', 'RBS0.25', 14.888088515698387, 179.1119114843016],
    'F3': ['B3', 'GFP10', 'RBS0.25', 14.888088515698387, 179.1119114843016],
    'C3': ['C3', 'GFP25', 'RBS0.25', 15.162434228225004, 178.837565771775],
    'B7': ['C3', 'GFP25', 'RBS0.25', 15.162434228225004, 178.837565771775],
    'C6': ['D3', 'GFP50', 'RBS0.25', 14.780178584485679, 179.21982141551433],
    'C8': ['D3', 'GFP50', 'RBS0.25', 14.780178584485679, 179.21982141551433],
    'C5': ['E3', 'GFP75', 'RBS0.25', 16.177011208889542, 177.82298879111045],
    'G2': ['E3', 'GFP75', 'RBS0.25', 16.177011208889542, 177.82298879111045],
    'B8': ['B4', 'GFP10', 'RBS0.5', 14.368139322107062, 179.63186067789294],
    'E4': ['B4', 'GFP10', 'RBS0.5', 14.368139322107062, 179.63186067789294],
    'F8': ['C4', 'GFP25', 'RBS0.5', 14.177042519250039, 179.82295748074995],
    'D8': ['C4', 'GFP25', 'RBS0.5', 14.177042519250039, 179.82295748074995],
    'E11': ['D4', 'GFP50', 'RBS0.5', 14.04122495885724, 179.95877504114276],
    'G8': ['D4', 'GFP50', 'RBS0.5', 14.04122495885724, 179.95877504114276],
    'B9': ['E4', 'GFP75', 'RBS0.5', 12.91243699667013, 181.08756300332988],
    'D5': ['E4', 'GFP75', 'RBS0.5', 12.91243699667013, 181.08756300332988],
    'F6': ['B5', 'GFP10', 'RBS1', 17.260539687949443, 176.73946031205057],
    'D4': ['B5', 'GFP10', 'RBS1', 17.260539687949443, 176.73946031205057],
    'C4': ['C5', 'GFP25', 'RBS1', 13.621200957373574, 180.37879904262644],
    'C7': ['C5', 'GFP25', 'RBS1', 13.621200957373574, 180.37879904262644],
    'E8': ['D5', 'GFP50', 'RBS1', 15.559824038689973, 178.44017596131002],
    'C9': ['D5', 'GFP50', 'RBS1', 15.559824038689973, 178.44017596131002],
    'E3': ['E5', 'GFP75', 'RBS1', 15.361074565601413, 178.63892543439857],
    'E2': ['E5', 'GFP75', 'RBS1', 15.361074565601413, 178.63892543439857],
    'F7': ['B6', 'GFP10', 'RBS2', 15.167336946557155, 178.83266305344284],
    'D2': ['B6', 'GFP10', 'RBS2', 15.167336946557155, 178.83266305344284],
    'G11': ['C6', 'GFP25', 'RBS2', 14.385767887360423, 179.61423211263957],
    'B4': ['C6', 'GFP25', 'RBS2', 14.385767887360423, 179.61423211263957],
    'F5': ['D6', 'GFP50', 'RBS2', 14.211407804175272, 179.78859219582472],
    'G9': ['D6', 'GFP50', 'RBS2', 14.211407804175272, 179.78859219582472],
    'D9': ['E6', 'GFP75', 'RBS2', 14.198501476331673, 179.80149852366833],
    'G3': ['E6', 'GFP75', 'RBS2', 14.198501476331673, 179.80149852366833],
    'F11': ['B7', 'GFP10', 'RBS4', 13.781297676024332, 180.21870232397566],
    'E6': ['B7', 'GFP10', 'RBS4', 13.781297676024332, 180.21870232397566],
    'D3': ['C7', 'GFP25', 'RBS4', 13.023589111341503, 180.9764108886585],
    'C2': ['C7', 'GFP25', 'RBS4', 13.023589111341503, 180.9764108886585],
    'B2': ['D7', 'GFP50', 'RBS4', 14.164197231724573, 179.8358027682754],
    'C11': ['D7', 'GFP50', 'RBS4', 14.164197231724573, 179.8358027682754],
    'B3': ['E7', 'GFP75', 'RBS4', 14.733746703467382, 179.26625329653262],
    'E10': ['E7', 'GFP75', 'RBS4', 14.733746703467382, 179.26625329653262],
    'C10': ['blank', 'blank', 'blank', 0, 194],
    'D10': ['blank', 'blank', 'blank', 0, 194],
    'E7': ['blank', 'blank', 'blank', 0, 194],
    'F2': ['blank', 'blank', 'blank', 0, 194],
    'F4': ['blank', 'blank', 'blank', 0, 194],
    'F9': ['blank', 'blank', 'blank', 0, 194],
    'F10': ['blank', 'blank', 'blank', 0, 194],
    

Now we add the dictionaries to the base protocol and test to make sure it runs.

In [26]:
# Make a copy of base_protocol.py and name it complete_protocol.py
os.system('cp base_protocol_multichannel.py complete_protocol.py')

# Add the volumes to the complete_protocol.py file
with open('complete_protocol.py', 'a') as file:
    file.write(f'final_positions = {final_positions}\n')
    file.write(f'iptg_volume = {iptg_volume}\n')

# Simulate the protocol
opentrons.simulate.simulate(open(protocol_file))

([{'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'location': H1 of Opentrons 96 Tip Rack 300 µL on 8,
    'text': 'Picking up tip from H1 of Opentrons 96 Tip Rack 300 µL on 8'},
   'logs': [<LogRecord: opentrons.calibration_storage.ot2.tip_length, 30, /home/croots/miniconda3/envs/opentrons/lib/python3.10/site-packages/opentrons/calibration_storage/ot2/tip_length.py, 53, "Tip length calibrations not found for None">]},
  {'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'volume': 195.26756169757405,
    'location': Location(point=Point(x=278.94, y=133.4, z=7.289999999999999), labware=A1 of USA Scientific 12 Well Reservoir 22 mL on 6),
    'rate': 1.0,
    'text': 'Aspirating 195.26756169757405 uL from A1 of USA Scientific 12 Well Reservoir 22 mL on 6 at 150.0 uL/sec'},
   'logs': []},
  {'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'volume': 177.51596517961275,
    'loca